In [1]:
from casadi import vertcat,SX
import sympy as sp
from sympy.physics.mechanics import dynamicsymbols

In [2]:
def sympy2casadi(sympy_expr,sympy_var,casadi_var):
    import casadi
    assert casadi_var.is_vector()
    if casadi_var.shape[1]>1:
        casadi_var = casadi_var.T
    casadi_var = casadi.vertsplit(casadi_var)
    from sympy.utilities.lambdify import lambdify

    mapping = {'ImmutableDenseMatrix': casadi.blockcat,
                'MutableDenseMatrix': casadi.blockcat,
                'Abs':casadi.fabs
            }
    f = lambdify(sympy_var, sympy_expr,modules=[mapping, casadi])
    print(casadi_var)
    return f(*casadi_var)

In [3]:
# Define the symbols
theta_sp, phi_sp = dynamicsymbols('theta phi')
# Create the matrix T
T = sp.Matrix([
    [1, sp.sin(phi_sp)*sp.tan(theta_sp), sp.cos(phi_sp)*sp.tan(theta_sp)],
    [0, sp.cos(phi_sp), -sp.sin(phi_sp)],
    [0, sp.sin(phi_sp)/sp.cos(theta_sp), sp.cos(phi_sp)/sp.cos(theta_sp)]
])


dtheta_sp = sp.diff(theta_sp,'t')
dphi_sp =sp.diff(phi_sp,'t')

dT_sp = sp.diff(T,'t',1)

dT_sp

Matrix([
[0,         (tan(theta(t))**2 + 1)*sin(phi(t))*Derivative(theta(t), t) + cos(phi(t))*tan(theta(t))*Derivative(phi(t), t),          (tan(theta(t))**2 + 1)*cos(phi(t))*Derivative(theta(t), t) - sin(phi(t))*tan(theta(t))*Derivative(phi(t), t)],
[0,                                                                                   -sin(phi(t))*Derivative(phi(t), t),                                                                                    -cos(phi(t))*Derivative(phi(t), t)],
[0, sin(phi(t))*sin(theta(t))*Derivative(theta(t), t)/cos(theta(t))**2 + cos(phi(t))*Derivative(phi(t), t)/cos(theta(t)), -sin(phi(t))*Derivative(phi(t), t)/cos(theta(t)) + sin(theta(t))*cos(phi(t))*Derivative(theta(t), t)/cos(theta(t))**2]])

In [5]:
theta = SX.sym("thet")
dtheta = SX.sym("dthet")
phi = SX.sym("phi")
dphi = SX.sym("dphi")
dT = sympy2casadi(dT_sp, [theta_sp, dtheta_sp, phi_sp, dphi_sp], vertcat(theta,dtheta,phi,dphi))
dT

[SX(thet), SX(dthet), SX(phi), SX(dphi)]


SX(@1=0, @2=1, 
[[@1, (((dthet*(sq(tan(thet))+@2))*sin(phi))+((dphi*cos(phi))*tan(thet))), (((dthet*(sq(tan(thet))+@2))*cos(phi))-((dphi*sin(phi))*tan(thet)))], 
 [@1, (-(dphi*sin(phi))), (-(dphi*cos(phi)))], 
 [@1, ((((dthet*sin(thet))*sin(phi))/sq(cos(thet)))+((dphi*cos(phi))/cos(thet))), ((((dthet*sin(thet))*cos(phi))/sq(cos(thet)))-((dphi*sin(phi))/cos(thet)))]])